In [1]:
import numpy as np
import json
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os
from src.src.uw_spider.fd import FD_Network


Step 1) Open the taget spider web with tension

In [2]:
with open(os.path.join('DATA', "public_spider_network.json"), "r") as f:
    data = json.load(f)
net = Network_custom.direct(np.array(data["vertices"]), data["edges"], data["q"], data["fixed"])
net.dir = np.ones(len(net.edges))
net.dir[::2] = -1

c:\Users\thijs\Documents\GitHub\printing-tensioned-structures\src\network.py:379: UserWarning: No paths provided. Make sure to set them later.
  Warning.warn("No paths provided. Make sure to set them later.")


Step 2) Determine goal edge lengths l0, and optimize vertices

In [3]:
file_path = os.path.join('DATA', 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path, A_scale = 1.0)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic') # In the paper l_scalar is called 's'

net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 100000, "damping": .1})
net.optimize_vertices()

Iteration 0: Current error = 2.638909057882832
Iteration 100: Current error = 0.27704168365311926
Iteration 200: Current error = 0.18954567060741637
Iteration 300: Current error = 0.15938203536439044
Iteration 400: Current error = 0.1426135828548656
Iteration 500: Current error = 0.1311336842809244
Iteration 600: Current error = 0.12245675501945363
Iteration 700: Current error = 0.11549050817609303
Iteration 800: Current error = 0.1097087127453257
Iteration 900: Current error = 0.10477280637394697
Iteration 1000: Current error = 0.10052621647017967
Iteration 1100: Current error = 0.09679987282001243
Converged after 1127 iterations.
Final error: 0.09589933757668961


Animate the Relaxation like this

In [4]:
%matplotlib qt
ani = net.ShapeOptimizer.animate_optimization()

Step 3) Scale the web with s (l_scalar)

In [5]:
reference_point = [0,0,0] 
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs=False) # If you don't provide a scalar, it will use network.l_scalar automatically. Note, that in this particular case, there are no leafs, so the last argument is not necessary.

#  You can plot the scaled network like this
# net.net_plot(color=True, plot_type='scaled')

# Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
xyz = net.arc_points(n = 9) 

net.net_plot(color=True, plot_type='arcs', elables=False)
net.l_scalar

0.9947809646201655